# 9 State-Space Models

State-space models define a rich class of processes that have served well as time series models. They contain ARIMA and SARIMA models as special cases.

## 9.1 State-Space Models

In this section, we shall illustrate some of the many time series models which can be represented in linear state-space form. We start with a more general definition of noise.

- $W_t \sim UN(\mu_t, R_t)$ denotes uncorrelated noise with mean vectors $\mu_t$ and covariance matrix $R_t$ (noting that they are allowed to change with $t$)
- $W_t \sim WN(\mu, R)$ denotes white noise as defined before.

The state-space model for $w$-dimensional time series $Y^1, Y^2, \ldots$, consists of two equations:

1. **Observation equation** takes the form:

   $$
   Y_t = G_t X_t + W_t, \quad t = 1, 2, \ldots \tag{9.1}
   $$

   where:

   - $X_t$ is a $v$-dimensional state vector (stochastic in general)
   - $Y_t$ is a $w$-dimensional output vector
   - $G_t$ is a $w \times v$ observation matrix (deterministic)
   - $W_t \sim UN(0, R_t)$ is observation noise, with $W_t$ and $R_t$ having dimensions $w$ and $w \times w$ (can be degenerate, i.e., $\det R_t = 0$)

   The observation equation essentially says that we can observe linear combinations of variables in the state vector, but only in the presence of noise.

2. **State-transition equation** takes the form:

   $$
   X_{t+1} = F_t X_t + V_t, \quad t = 1, 2, \ldots \tag{9.2}
   $$

   where:

   - $F_t$ is a $v \times v$ state transition matrix (deterministic)
   - $V_t \sim UN(0, Q_t)$ is state transition noise, with $V_t$ and $Q_t$ having dimensions $v$ and $v \times v$

   Recursively, we have:

   $$
   \begin{align*}
   X_t &= F_{t-1}(F_{t-2} X_{t-2} + V_{t-2}) + V_{t-1} \\
       &= \cdots \\
       &= (F_{t-1} \cdots F_1)X_1 + (F_{t-1} \cdots F_2)V_1 + \cdots + F_{t-1}V_{t-2} + V_{t-1} \\
       &= f_t(X_1, V_1, \ldots, V_{t-1})
   \end{align*}


We also have:

$$
Y_t = G_t f_t(X_1, V_1, \ldots, V_{t-1}) + W_t \\
= g_t(X_1, V_1, \ldots, V_{t-1}, W_t)
$$

### Two Additional Assumptions

(a) $E\{W_s V_t^\top\} = 0$ for all $s$ and $t$  
(here $0$ is a $w \times v$ matrix of zeros; in words, every observation noise random variable is uncorrelated with every state-transition noise random variable).

(b) Assuming $E\{X_t\} = 0$ for convenience, then $E\{X_1 W_t^\top\} = 0$ and $E\{X_1 V_t^\top\} = 0$ for all $t$  
(in words, initial state vector random variables are uncorrelated with observation and state-transition noise).

---

### Example 9.1

To interpret the state-space model, we start with a very simple example: recall the classical decomposition model for time series $Y_t$, namely,

$$
Y_t = m_t + s_t + W_t
$$

where:

- $m_t$ is trend,
- $s_t$ is periodic,
- $W_t$ is a stationary process.

$m_t$ and $s_t$ can be treated as deterministic or stochastic.

Now, we consider the **simple version**, known as a **local level model**, in which $m_t$ is stochastic and $s_t = 0$:

$$
Y_t = X_t + W_t, \quad \{W_t\} \sim WN(0, \sigma^2_W)
$$

$$
X_{t+1} = X_t + V_t, \quad \{V_t\} \sim WN(0, \sigma^2_V)
$$

with the assumption:

$$
E\{W_s V_t\} = 0 \quad \text{for all } s, t
$$

This can be easily seen as a special case of a state-space model:

**Observation equation:**

$$
Y_t = G_t X_t + W_t
$$

which simplifies to:

$$
Y_t = X_t + W_t
$$

with:

- $Y_t = Y_t$,
- $G_t = 1$,
- $X_t = X_t$,
- $W_t = W_t$,
- $R_t = \sigma^2_W$

**State-transition equation:**

$$
X_{t+1} = F_t X_t + V_t \tag{9.3}
$$

which simplifies to:

$$
X_{t+1} = X_t + V_t \tag{9.4}
$$

with:

- $X_{t+1} = X_{t+1}$,
- $F_t = 1$,
- $V_t = V_t$,
- $Q_t = \sigma^2_V$

To fully specify this state-space model, we need to define the initial state $X_1 = X_t$ as a random variable that is uncorrelated with the $W_t$'s and $V_t$'s. Additionally, we assume:

- $E(X_t) = m_1$
- $\text{Var}(X_t) = P_1$

Thus, this model has **4 parameters**:

- $\sigma^2_W = R_t$
- $\sigma^2_V = Q_t$
- $m_1$
- $P_1$

Since $X_{t+1} = X_t + V_t$, the state variable $X_t$ is a **random walk** starting from $m_1$, and then $Y_t$ is the sequence of $X_t$ corrupted by noise $W_t$.


In [ ]:
# Initial state parameters
m1 <- 1        # Mean of X1
P1 <- 1        # Variance of X1
SigmaV <- 2    # Standard deviation of process noise

# Simulate initial state
X1 <- rnorm(1, mean = m1, sd = sqrt(P1))

# Number of time steps
N <- 100

par(bg = "white")        

# Simulate process noise Vt (length N-1)
Vt <- rnorm(N - 1, mean = 0, sd = SigmaV)

# Generate latent state sequence Xt as a random walk starting from X1
Xt <- cumsum(c(X1, Vt))

# Loop over different observation noise levels (SigmaW)
for (SigmaW in c(0, 1, 5)) {
  
  # Simulate observed data Yt with additive Gaussian noise
  Yt <- rnorm(N, mean = Xt, sd = SigmaW)
  
  # Plot observed data (Yt in blue) and true state (Xt in red)
  plot(1:N, Yt, col = "blue", type = "p", 
       xlab = "t", ylab = "Yt in blue, Xt in red", main = paste("SigmaW =", SigmaW))
  lines(1:N, Xt, col = "red")
}


### Example 9.2: AR(1) Process as a State-Space Model

The AR(1) model has the form:

$$
X_{t+1} = \phi X_t + W_{t+1}, \quad \{W_t\} \sim WN(0, \sigma^2)
$$

This provides:

$$
X_t = \sum_{j=0}^{\infty} \phi^j W_{t-j}
$$

and consequently:

$$
X_1 = \sum_{j=0}^{\infty} \phi^j W_{1-j}
$$

We can write this in the state-space model form as:

- $X_{t+1} = X_{t+1}$
- $F_t = \phi$
- $V_t = W_{t+1}$
- $Y_t = X_t$
- $G_t = 1$
- $W_t = 0$

---

### Example 9.3: ARMA(1,1) as a State-Space Model

Let $\{X_t\}$ be the causal and invertible ARMA(1,1) process satisfying:

$$
X_t = \phi X_{t-1} + W_t + \theta W_{t-1}, \quad \{W_t\} \sim WN(0, \sigma^2)
$$

To write this in state-space model form, we first observe:

$$
X_t = (1 - \phi B)^{-1}(1 + \theta B)W_t \\
= (1 + \theta B)\left\{(1 - \phi B)^{-1}W_t\right\} \\
= (1 + \theta B)Z_t
$$

where $Z_t = (1 - \phi B)^{-1}W_t$. That is,

$$
(1 - \phi B)Z_t = W_t \quad \Rightarrow \quad Z_t = \phi Z_{t-1} + W_t
$$

We can write this as a vector system:

$$
\begin{bmatrix}
Z_t \\
Z_{t+1}
\end{bmatrix}
=
\begin{bmatrix}
0 & 1 \\
0 & \phi
\end{bmatrix}
\begin{bmatrix}
Z_{t-1} \\
Z_t
\end{bmatrix}
+
\begin{bmatrix}
0 \\
W_{t+1}
\end{bmatrix}
$$

Thus, this is a **state-space representation** of the ARMA(1,1) process.


Further, since:

$$
Z_t = \sum_{j=0}^{\infty} \phi^j W_{t-j}
$$

we can express:

$$
X_{t+1} =
\begin{bmatrix}
Z_t \\
Z_{t+1}
\end{bmatrix}
$$

with:

- $F_t =
\begin{bmatrix}
0 & 1 \\
0 & \phi
\end{bmatrix}$

- $V_t =
\begin{bmatrix}
0 \\
W_{t+1}
\end{bmatrix}$

- $Y_t = X_t$
- $G_t = \begin{bmatrix} \theta & 1 \end{bmatrix}$
- $W_t = 0$

and the initial state:

$$
X_1 =
\begin{bmatrix}
Z_0 \\
Z_1
\end{bmatrix}
=
\begin{bmatrix}
\sum_{j=0}^{\infty} \phi^j W_{-j} \\
\sum_{j=0}^{\infty} \phi^j W_{1-j}
\end{bmatrix}
$$

---

### Example 9.4: AR(p) as a State-Space Model

Consider the AR(p) process:

$$
X_t = \phi_1 X_{t-1} + \cdots + \phi_p X_{t-p} + W_t, \quad \{W_t\} \sim WN(0, \sigma^2)
$$

Then, we can write:

$$
X_t = (X_t)^0
$$

_(Note: This is likely an abbreviated notation — let me know if you want the full matrix form of the AR(p) state-space representation.)_

---

### Four Classical Problems in State-Space Models

Given observations $Y_1, \ldots, Y_t$, we are typically interested in solving the following problems:

1. **Filtering**: What is the best predictor of state $X_t$?
2. **Forecasting**: What is the best predictor of state $X_{t+1}$?
3. **Smoothing**: What is the best predictor of state $X_s$ for $s < t$?
4. **Estimation**: What are the best estimates of model parameters?
